In [ ]:
import os
import cv2
import numpy as np

from torch.utils.data import DataLoader

import argus
from argus import Model
from argus.callbacks import MonitorCheckpoint, EarlyStopping
from argus.callbacks import LoggingToFile, ReduceLROnPlateau

from src.datasets import DrawDataset, get_train_val_samples
from src.transforms import ImageTransform, DrawTransform
from src.argus_models import CnnFinetune
from src.metrics import MAPatK
from src import config

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
def imshow(image, figsize=(3, 3)):
    plt.figure(figsize=figsize)
    plt.imshow(image)
    plt.show()

In [ ]:
image_size = 128
scale_size = 64
random_crop_size = (120, 128)
rotate_angle = 15
max_border_scale = 0.7
image_pad = 4
image_line_width = 2
time_color = False
train_batch_size = 2048
val_batch_size = 2048
train_epoch_size = 1000000
val_key_id_path = '/workdir/data/val_key_ids_001.json'

# Data

In [ ]:
train_samples, val_samples = get_train_val_samples(val_key_id_path)

In [ ]:
draw_transform = DrawTransform(image_size, image_pad, image_line_width, time_color)
train_trns = ImageTransform(True, scale_size, random_crop_size, rotate_angle, max_border_scale)
train_dataset = DrawDataset(train_samples, draw_transform,
                            size=train_epoch_size, image_transform=train_trns)
val_trns = ImageTransform(False)
val_dataset = DrawDataset(val_samples, draw_transform, image_transform=val_trns)

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, num_workers=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=val_batch_size, num_workers=16, shuffle=False)

In [ ]:
n_images_to_draw = 3

for img, trg in train_loader:
    for i in range(n_images_to_draw):
        img_i = img[i, 0, :, :].numpy()
        print(config.IDX_TO_CLASS[trg[i].item()])
        imshow(img_i)
    break

# Model

In [ ]:
params = {
    'nn_module': {
        'model_name': 'resnet34',
        'num_classes': len(config.CLASSES),
        'pretrained': True,
        'dropout_p': 0.5
    },
    'optimizer': ('Adam', {'lr': 0.001}),
    'loss': 'CrossEntropyLoss',
    'device': 'cuda'
}

model = CnnFinetune(params)

In [ ]:
experiment_name = 'cnn_fine_resnet34_006'

callbacks = [
    MonitorCheckpoint(f'/workdir/data/experiments/{experiment_name}', monitor='val_map_at_k', max_saves=3),
    EarlyStopping(monitor='val_map_at_k', patience=50),
    ReduceLROnPlateau(monitor='val_map_at_k', factor=0.64, patience=5, min_lr=0.00001),
    LoggingToFile(f'/workdir/data/experiments/{experiment_name}/log.txt')
]

model.fit(train_loader, 
          val_loader=val_loader,
          max_epochs=1000,
          callbacks=callbacks,
          metrics=['accuracy', MAPatK(k=3)])

# Predict 

In [ ]:
import torch


class Predictor:
    def __init__(self, model_path, draw_transform, image_transform):
        self.model = load_model(model_path)
        self.model.nn_module.eval()

        self.draw_transform = draw_transform
        self.image_transform = image_transform

    def __call__(self, drawings):
        tensors = []
        for drawing in drawings:
            image = self.draw_transform(drawing)
            tensor = self.image_transform(image)
            tensors.append(tensor)
        
        tensor = torch.stack(tensors, dim=0)
        tensor = tensor.to(self.model.device)

        with torch.no_grad():
            probs = self.model.predict(tensor)
            return probs


In [ ]:
import pandas as pd
from src import config
import tqdm

from argus import load_model

pred_batch_size = 1024 
model_path = '/workdir/data/experiments/cnn_fine_se_resnext50_004/model-097-0.814777.pth'

test_df = pd.read_csv(config.TEST_SIMPLIFIED_PATH)
sample_subm = pd.read_csv(config.SAMPLE_SUBMISSION)
predictor = Predictor(model_path, draw_transform, val_trns)

drawings = []
key_ids = []
pred_words = []
pred_key_ids = []
for i, row in tqdm.tqdm(test_df.iterrows()):
    drawing = eval(row.drawing)
    
    drawings.append(drawing)
    key_ids.append(row.key_id)
    if len(drawings) == pred_batch_size:
        probs = predictor(drawings).cpu().numpy()
        preds_idx = probs.argsort(axis=1)
        preds_idx = np.fliplr(preds_idx)[:, :3]
        for pred_idx, key_id in zip(preds_idx, key_ids):
            words = [config.IDX_TO_CLASS[i].replace(' ', '_') for i in pred_idx]
            pred_words.append(" ".join(words))
            pred_key_ids.append(key_id)
            
        drawings = []
        key_ids = []
        
probs = predictor(drawings).cpu().numpy()
preds_idx = probs.argsort(axis=1)
preds_idx = np.fliplr(preds_idx)[:, :3]
for pred_idx, key_id in zip(preds_idx, key_ids):
    words = [config.IDX_TO_CLASS[i].replace(' ', '_') for i in pred_idx]
    pred_words.append(" ".join(words))
    pred_key_ids.append(key_id)

drawings = []
key_ids = []
    
subm = pd.DataFrame({'key_id': pred_key_ids, 'word': pred_words})

In [ ]:
subm.to_csv('/workdir/data/first_subm.csv', index=False)